In [3]:
import jax
import jax.numpy as jnp
import numpy as np

from jax.experimental import loops
from jax.ops import index_update, index_add

2022-02-14 10:18:09.297477: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ba2617/code-server-3.12.0-linux-amd64/:
2022-02-14 10:18:10.933940: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ba2617/code-server-3.12.0-linux-amd64/:
2022-02-14 10:18:10.944823: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ba2617/code-server-3.12.0-linux-amd64/:


In [28]:
def pava(X, Y):
    # maybe add shape checking here...
    idx = jnp.argsort(X)
    X_s = X[idx]
    Y_s = Y[idx]
    
    X_unique = jnp.unique(X_s)
    num_unique = len(X_unique)
    X_unique, Y_unique = make_unique_fixed(X_s, Y_s, X_unique, num_unique)
    return X_unique, Y_unique
    
def _pava(X_unique, Y_unique):
    pass
    
def make_unique_fixed(X, Y, X_unique, num_unique):
    # vmap-friendly version of make_unique_flexible.
    # Gets rid of duplicate independent variables in X,
    # and averages the corresponding obervations in Y.
    # NOTE: Assumes X and Y are sorted.
    
    with loops.Scope() as s:
        s.Y_out = jnp.empty(num_unique)
        
        for i in s.range(num_unique):
            curr_val = X_unique[i]
            avg_y = jnp.nanmean(jnp.where(X == curr_val, Y, jnp.nan))
            s.Y_out = index_update(s.Y_out, i, avg_y)
            
        return X_unique, s.Y_out
    
def make_unique_flexible(X, Y, eps=1e-8):
    # Get rid of duplicate independent variables and associated observations
    # by averaging. In this case, we consider independent vars. X within some
    # tolerance to be the same.
    # 
    # This is a direct copy of the implementation from sklearn here:
    # https://github.com/scikit-learn/scikit-learn/blob/7e1e6d09bcc2eaeba98f7e737aac2ac782f0e5f1/sklearn/_isotonic.pyx#L66
    # but modified to assume that all weights are equal 1
    n_unique = len(jnp.unique(X))
    n_samples = X.shape[0]
    
    
    with loops.Scope() as s:
        
        s.X_out = jnp.empty(n_unique)
        s.Y_out = jnp.empty_like(s.X_out)

        s.w_cur = 0
        s.x_cur = X[0]
        s.y_cur = 0
        s.i = 0
        s.x = 0
        
        def true_fun(args):
            i, w_cur, j, X_out, Y_out, x, x_cur, y_cur = args
            X_out = index_update(X_out, i, x_cur)
            Y_out = index_update(Y_out, i, y_cur / w_cur)
            x_cur = x
            y_cur = Y[j]
            i += 1
            w_cur = 1
            return i, w_cur, X_out, Y_out, x_cur, y_cur
            
        def false_fun(args):
            i, w_cur, j, X_out, Y_out, x, x_cur, y_cur = args
            y_cur += Y[j]
            w_cur += 1
            return i, w_cur, X_out, Y_out, x_cur, y_cur
    
        for j in s.range(n_samples):
            s.x = X[j]
            
            s.i, s.w_cur, s.X_out, s.Y_out, s.x_cur, s.y_cur = jax.lax.cond(
                s.x - s.x_cur >= eps,
                true_fun,
                false_fun,
                (s.i, s.w_cur, j, s.X_out, s.Y_out, s.x, s.x_cur, s.y_cur),
            )
        
        s.X_out = index_update(s.X_out, s.i, s.x_cur)
        s.Y_out = index_update(s.Y_out, s.i, s.y_cur / s.w_cur)
        
        return s.X_out[:s.i+1], s.Y_out[:s.i+1]
    

    

In [29]:
x = jnp.array([25,25,25,30,30,30])
y = jnp.array([10,12,1,23, 19, 7])

pava(x,y)

(DeviceArray([25, 30], dtype=int32),
 DeviceArray([ 7.6666665, 16.333334 ], dtype=float32))

In [62]:
def f1(args):
    a,b,c = args
    return c

def f2(args):
    a,b,c = args
    return a

a = 1
b = 3
c = 1
jax.lax.cond(
    a > b,
    f1,
    f2,
    (a,b,c)
)

DeviceArray(1, dtype=int32)